In [2]:
import mlflow
from lib.reproduction import training_info
from pathlib import Path
import pickle

mlflow.set_tracking_uri("http://localhost:5000")

/home/christian/projects/p9/baseline/venv/lib/python3.10/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/christian/projects/p9/baseline/venv/lib/python3.10/site-packages/pydantic/_internal/_config.py:321: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [3]:
client = mlflow.MlflowClient()

kv = {}
# for oxide, v in training_info.items():
#     sub = {}
#     for comp_range_name, vv in v.items():
#         model_name = f"{oxide}_{comp_range_name}_outlier_removal"
#         models = client.search_model_versions(f"name = '{model_name}'")
#         model = mlflow.pyfunc.load_model(models[0].source)
#         sub[comp_range_name] = model
        
#     kv[oxide] = sub


models_path = Path("../models/PLS_Models_12-11-23_174732/")
for oxide, v in training_info.items():
    sub = {}
    for comp_range_name, vv in v.items():
        model_name = f"{oxide}_{comp_range_name}/model.pkl"
        model = pickle.load(open(models_path / model_name, "rb"))
        sub[comp_range_name] = model

    kv[oxide] = sub


kv

{'SiO2': {'Full': PLSRegression(n_components=6),
  'Low': PLSRegression(n_components=9),
  'Mid': PLSRegression(n_components=6),
  'High': PLSRegression(n_components=5)},
 'TiO2': {'Full': PLSRegression(n_components=5),
  'Low': PLSRegression(n_components=7),
  'Mid': PLSRegression(n_components=5),
  'High': PLSRegression(n_components=3)},
 'Al2O3': {'Full': PLSRegression(n_components=6),
  'Low': PLSRegression(n_components=6),
  'Mid': PLSRegression(n_components=8),
  'High': PLSRegression(n_components=6)},
 'FeOT': {'Full': PLSRegression(n_components=8),
  'Low': PLSRegression(n_components=3),
  'Mid': PLSRegression(n_components=8),
  'High': PLSRegression(n_components=3)},
 'MgO': {'Full': PLSRegression(n_components=7),
  'Low': PLSRegression(n_components=6),
  'Mid': PLSRegression(n_components=9),
  'High': PLSRegression(n_components=8)},
 'CaO': {'Full': PLSRegression(n_components=8),
  'Low': PLSRegression(n_components=9),
  'Mid': PLSRegression(n_components=9),
  'High': PLSRegr

In [4]:
import dotenv
from lib.data_handling import load_data
env = dotenv.dotenv_values(dotenv.find_dotenv())

dataset_loc = env.get('DATA_PATH') or ""
print(dataset_loc)

smol_data = load_data(
    "../data/data/calib/calib_2015/1600mm/pls/",
    10
)

data/data/calib/calib_2015/1600mm/pls


Loading data:   0%|          | 0/10 [00:00<?, ?it/s]

Loading data: 100%|██████████| 10/10 [00:05<00:00,  1.77it/s]


In [5]:
from lib.data_handling import CustomSpectralPipeline
from lib.reproduction import masks, major_oxides

compositon_data_path = env.get('COMPOSITION_DATA_PATH') or ""
print(compositon_data_path)

pipeline = CustomSpectralPipeline(
    masks,
    "../data/data/calib/ccam_calibration_compositions.csv",
    major_oxides
)

processed_data = pipeline.fit_transform(smol_data)
processed_data.head()

data/data/calib/ccam_calibration_compositions.csv


Transforming samples:   0%|          | 0/10 [00:00<?, ?it/s]

Transforming samples: 100%|██████████| 10/10 [00:00<00:00, 13.42it/s]


,246.688,246.741,246.79401,246.847,246.89999,246.953,247.007,247.06,247.11301,247.166,...,848.89642,SiO2,TiO2,Al2O3,FeOT,MgO,CaO,Na2O,K2O,Sample Name
0,3.998589e+11,3.623102e+11,3.125834e+11,2.548143e+11,2.384109e+11,2.535727e+11,2.867055e+11,3.181519e+11,3.228308e+11,3.108220e+11,...,2.293541e+10,51.03,1,13.83,9.93,6.33,10.7,2.01,0.6,jsc1399
1,3.101115e+11,2.760840e+11,2.294153e+11,1.789714e+11,1.602546e+11,1.737365e+11,2.033433e+11,2.307903e+11,2.380900e+11,2.286484e+11,...,2.224658e+10,51.03,1,13.83,9.93,6.33,10.7,2.01,0.6,jsc1399
2,4.784395e+11,4.364859e+11,3.787261e+11,3.007351e+11,2.869929e+11,3.060502e+11,3.437366e+11,3.862550e+11,3.896103e+11,3.750934e+11,...,2.239592e+10,51.03,1,13.83,9.93,6.33,10.7,2.01,0.6,jsc1399
3,5.402266e+11,4.974165e+11,4.266432e+11,3.354670e+11,3.232402e+11,3.433198e+11,3.847175e+11,4.298031e+11,4.313859e+11,4.173089e+11,...,1.634240e+09,51.03,1,13.83,9.93,6.33,10.7,2.01,0.6,jsc1399
4,4.925927e+11,4.457917e+11,3.862441e+11,3.063318e+11,2.987273e+11,3.191648e+11,3.567172e+11,3.909940e+11,3.978301e+11,3.836622e+11,...,2.234164e+10,51.03,1,13.83,9.93,6.33,10.7,2.01,0.6,jsc1399


In [19]:
from lib.norms import Norm1Scaler, Norm3Scaler
from lib.reproduction import spectrometer_wavelength_ranges
import lib.norms
import importlib
importlib.reload(lib.norms)

drop_cols = major_oxides + ['Sample Name']

scaler = Norm1Scaler(reshaped=True)
X_1 = scaler.fit_transform(processed_data.drop(drop_cols, axis=1))
X_3 = Norm3Scaler(spectrometer_wavelength_ranges, reshaped=True).fit_transform(processed_data.drop(drop_cols, axis=1))

y = processed_data['SiO2']

In [7]:
from sklearn.metrics import mean_squared_error
predictions = model.predict(X[:1])
predictions
# out = mean_squared_error(predictions, y[:1], squared=False)
# out/100


/home/christian/projects/p9/baseline/venv/lib/python3.10/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but PLSRegression was fitted without feature names
  warnings.warn(


array([1.43265354])

In [23]:
def get_weights(y_full, blend_range_min, blend_range_max):
    """
    Helper function to calculate weights for blending predictions.
    """
    w_upper = (y_full - blend_range_min) / (blend_range_max - blend_range_min)
    w_lower = 1 - w_upper
    return w_lower, w_upper


def predict_composition_with_blending(oxide, y_full, X1, X3, models, optimized_blending_ranges):
    """
    Predicts the composition of the given oxide based on the full model prediction (y_full)
    and the optimized blending ranges, including blending between "mid" and "high" models as well as "low" and "mid" models.
    """
    # Check for non-blending range predictions first
    blend_ranges = ["Low-Mid", "Mid-High"]

    for range_name, (range_min, range_max) in optimized_blending_ranges[oxide].items():
        if range_min <= y_full <= range_max and range_name not in blend_ranges:
            return models[oxide][range_name].predict(X1 if training_info[oxide][range_name]["normalization"] == 1 else X3)

    for blend_range in blend_ranges:
        # Check if blend_range is defined for the given oxide
        if blend_range in optimized_blending_ranges[oxide]:
            blend_range_min, blend_range_max = optimized_blending_ranges[oxide][blend_range]
            print(f"Range: {blend_range}, min: {blend_range_min}, max: {blend_range_max}")

            # Check if y_full is within the defined blending range
            if blend_range_min <= y_full <= blend_range_max:
                w_lower, w_upper = get_weights(y_full, blend_range_min, blend_range_max)

                lower_norm = training_info[oxide][blend_range.split("-")[0]]["normalization"]
                upper_norm = training_info[oxide][blend_range.split("-")[1]]["normalization"]

                y_lower = models[oxide][blend_range.split("-")[0]].predict(X1 if lower_norm == 1 else X3)
                y_upper = models[oxide][blend_range.split("-")[1]].predict(X1 if upper_norm == 1 else X3)

                y_final = w_lower * y_lower + w_upper * y_upper
                print(f"y_lower: {y_lower}, y_upper: {y_upper}, y_final: {y_final}")
                print(f"w_lower: {w_lower}, w_upper: {w_upper}")

                return y_final

    # Error if y_full is outside any defined range
    raise ValueError(f"y_full value {y_full} for oxide {oxide} is outside the defined blending ranges.")

In [24]:
from lib.reproduction import optimized_blending_ranges  # noqa: E402

full_norm = training_info['SiO2']['Full']['normalization']

y_full = kv['SiO2']['Full'].predict(X_1[:1] if full_norm == 1 else X_3[:1])
print(y_full)
# kv['SiO2']['Low'].predict(X[:1])
predict_composition_with_blending('SiO2', y_full, X_1[:1], X_3[:1], kv, optimized_blending_ranges)

/home/christian/projects/p9/baseline/venv/lib/python3.10/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but PLSRegression was fitted without feature names
  warnings.warn(


[87.16749613]


/home/christian/projects/p9/baseline/venv/lib/python3.10/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but PLSRegression was fitted without feature names
  warnings.warn(


array([42.55337145])